## Introduction

Sust Global's climate scenario analysis datasets allow operational risk analysts, quantitative modellers and ESG analysts to assess historic, near term and long term impacts of climate change. We provide historic risk exposure scores for fires, floods and cyclones over the past 11 years (Jan 2010- Dec 2020). For forward looking projections of extreme physical climate hazards, we provide risk exposures across standardized climate scenarios. In this User Guide, we give an overview of the outputs from climate scenario analysis and describe the hazards that we are covering in our analysis. We describe the different views in our platform and provide guidance on interpreting the results from our climate scenario analysis. 

### Physical Hazard Modeling 

We model rising temperatures and varying precipitation at the asset level.  We report on 5 physical hazards: Wildfire, floods, heatwaves, droughts (SPEI) and sea level rise (SLR). The modelled results are stored as time series for each specific asset within the portfolio of assets.

### References

More details on our dataset, descriptio of climate scenarios and risk modalities and value ranges can be found in our [Climate Scenario Analysis User Guide](https://drive.google.com/file/d/1ksBEYJexyLafInkZzEbcLnb3Kuc5eVsx/view) which also outlines a collection of technical references. 

Visit the [Sust Global's website](https://www.sustglobal.com/financial-services) to learn more about our unique climate risk analytics capabilities.



In [ ]:
import json
import plotly.express as px
import pandas as pd
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import chart_studio.plotly as py

## Parameters

`DATASET_ROOT` folder indicates the parent folder of the collections folder. Kindly change this to reflect the right folder on your local machine. Place the multiple climate scenario analysis collections from Sust Global for your different portfolios/asset collections in this parent folder. `US_ZIPCODES` with it's 73 csv files would be a folder in this parent folder/

`START_YEAR` indicates the beginning of the analysis window

`END_YEAR` indicates the conclusion of the analysis window


In [ ]:
DATASET_ROOT = '../../data/'
START_YEAR = 2010
END_YEAR = 2100
START_DATE_HISTORIC = '20100101'
FLOODING_THRESHOLD = 0.05
STEP_YEAR = 10
EPSILON = 1e-8
max_risk_values = {'fire':6.0, 
                'flood':1.0, 
                'heatwaves':200.0, #100.00
                'SPEI':3.0, 
                'SLR':0.75, #1.5,
                'Cyclone':1.3,
                'combined':1.0}
low_risk_breakpoint = {'fire':2.0, 
                'flood':0.0, 
                'heatwaves':30.0,
                'SPEI':1.5, 
                'SLR':0.1,
                'Cyclone':0.6,
                'combined':0.5}
high_risk_breakpoint = {'fire':4.0, 
                'flood':3.0, 
                'heatwaves':50.0,
                'SPEI':2.0, 
                'SLR':0.3,
                'Cyclone':0.79,
                'combined':0.7}
colors = {
    'background': '#FFFFFF',
    'text': '#AAAAAA'
}

PORTFOLIO_FRACTION_EXPOSED = 0.4 

## Converting flood risk probabilities to decadal flood exceedance

In [ ]:
def transform_flood_probability_to_time_series(df_filtered, flooding_threshold):
    df_filtered.reset_index(drop=True, inplace=True)
    for col in df_filtered.columns:
        if df_filtered.at[0, col] > flooding_threshold:
            df_filtered.at[0, col] = 1.0
        else:
            df_filtered.at[0, col] = 0.0
    for year in range(START_YEAR, END_YEAR, STEP_YEAR):
        acc_floor_years = 0.0
        for inc_year in range(STEP_YEAR):
            acc_floor_years = acc_floor_years + df_filtered.at[0,str(year+inc_year)]
        for inc_year in range(STEP_YEAR):
            df_filtered.at[0,str(year+inc_year)] = acc_floor_years/STEP_YEAR
    return df_filtered

## Display climate scenario analysis time series data

In [ ]:
def display_timeseries_data(asset_index, selected_ssp, selected_portfolio):

    upper_bound_collection =[]
    lower_bound_collection = []
    trace_collection = []

    for selected_risk in ['fire', 'flood', 'heatwaves', 'SPEI', 'SLR']:

        data_file_asset_ssp585 = DATASET_ROOT+selected_portfolio+'/sustglobal_asset_fwd_' + \
            selected_risk+'_risk_'+selected_portfolio+'_'+selected_ssp+'.csv'
        data_file_asset_ssp585_lbd = DATASET_ROOT+selected_portfolio+'/sustglobal_asset_fwd_' + \
            selected_risk+'_risk_'+selected_portfolio+'_'+selected_ssp+'_lbd.csv'
        data_file_asset_ssp585_ubd = DATASET_ROOT+selected_portfolio+'/sustglobal_asset_fwd_' + \
            selected_risk+'_risk_'+selected_portfolio+'_'+selected_ssp+'_ubd.csv'

        df_asset_ssp585 = pd.read_csv(data_file_asset_ssp585)
        df_asset_ssp585_lbd = pd.read_csv(data_file_asset_ssp585_lbd)
        df_asset_ssp585_ubd = pd.read_csv(data_file_asset_ssp585_ubd)

        risk_colnames = [str(i) for i in list(range(START_YEAR, END_YEAR+1))]
        df_selected = df_asset_ssp585[risk_colnames]
        df_selected_lbd = df_asset_ssp585_lbd[risk_colnames]
        df_selected_ubd = df_asset_ssp585_ubd[risk_colnames]

        # attending to the case where you are switching portfolios and starting 
        # with an asset index greater that the size of the asset risk collection
        if asset_index > df_selected.shape[0]:
            asset_index = 0

        df_filtered = df_selected[asset_index:asset_index+1].copy()
        df_filtered_lbd = df_selected_lbd[asset_index:asset_index+1]
        df_filtered_ubd = df_selected_ubd[asset_index:asset_index+1]

        year_labels = [x for x in df_filtered.columns.values]
        year_val = pd.to_numeric(year_labels)

        if selected_risk == 'flood':
            df_filtered = transform_flood_probability_to_time_series(df_filtered, FLOODING_THRESHOLD)
            df_filtered_lbd = transform_flood_probability_to_time_series(df_filtered_lbd, FLOODING_THRESHOLD+0.02)
            df_filtered_ubd = transform_flood_probability_to_time_series(df_filtered_ubd, FLOODING_THRESHOLD-0.02)

        if selected_risk == 'SPEI':
            df_filtered = df_filtered*-1.0
            df_filtered_lbd = df_filtered_lbd*-1.0
            df_filtered_ubd = df_filtered_ubd*-1.0

        risk_val = df_filtered.values[0]/max_risk_values[selected_risk]
        risk_lbd_val = df_filtered_lbd.values[0]/(max_risk_values[selected_risk]*4.0)
        risk_ubd_val = df_filtered_ubd.values[0]/(max_risk_values[selected_risk]*4.0)

        trace_color = {'fire':'rgb(255, 0, 0)', 
                       'flood':'rgb(0, 255, 255)', 
                       'heatwaves':'rgb(153, 0, 0)', 
                       'SPEI':'rgb(255, 123, 0)', 
                       'SLR':'rgb(0, 0, 255)'}

        upper_bound = go.Scatter(
            name='',
            x=year_val,
            y=risk_ubd_val,
            mode='lines',
            line=dict(width=0.5, color="rgba(255, 255, 255, 0)"),
            showlegend=False)

        trace = go.Scatter(
            name=selected_risk,
            x=year_val,
            y=risk_val,
            mode='lines',
            line=dict(width=3.0, color=trace_color[selected_risk]),
            fillcolor='rgba(100, 100, 100, 0.2)',
            fill='tonexty')

        lower_bound = go.Scatter(
            name='',
            x=year_val,
            y=risk_lbd_val,
            mode='lines',
            line=dict(width=0.5, color="rgba(255, 255, 255, 0)"),
            showlegend=False,
            fillcolor='rgba(100, 100, 100, 0.2)',
            fill='tonexty')

        upper_bound_collection.append(upper_bound)
        trace_collection.append(trace)
        lower_bound_collection.append(lower_bound)

    risk_label = 'Risk Exposure'
    data = []
    for trace_index in range(len(upper_bound_collection)):
        data.append(upper_bound_collection[trace_index])
        data.append(trace_collection[trace_index])
        data.append(lower_bound_collection[trace_index])

    df_asset = pd.read_csv(DATASET_ROOT+selected_portfolio+'/sustglobal_asset_fwd_' + \
            'fire'+'_risk_'+selected_portfolio+'_'+selected_ssp+'.csv')
    address_label = df_asset[asset_index:asset_index+1]['Address'].values[0]
    print("Address : ", address_label)
    
    layout = go.Layout(
        title='Normalized Multi-Hazard Predictive Risk Exposure : '+str(address_label),
        yaxis=dict(title=risk_label,
                   showline=True,
                   showgrid=False,
                   showticklabels=True,
                   linecolor='rgb(82, 82, 82)',
                   linewidth=2,
                   ticks='outside',
                   range=[0,1.1],
                   tickfont=dict(
                         family='Arial',
                         size=12,
                         color='rgb(82, 82, 82)')
                   ),
        xaxis=dict(title='Year of Assessment',
                   showline=True,
                   showgrid=False,
                   showticklabels=True,
                   linecolor='rgb(82, 82, 82)',
                   linewidth=2,
                   ticks='outside',
                   tickfont=dict(
                         family='Arial',
                         size=12,
                         color='rgb(82, 82, 82)'),
                   rangeselector=dict(
                       buttons=list([
                           dict(count=1,
                                label='1y',
                                step='year',
                                stepmode='backward'),
                           dict(count=3,
                                label='3y',
                                step='year',
                                stepmode='backward'),
                           dict(count=6,
                                label='10y',
                                step='year',
                                stepmode='backward'),
                           dict(step='all')
                       ])
                   ),
                   rangeslider=dict(
                       visible=True
                   ),
                   type='date'),
        showlegend=True,
        legend=dict(x=0, y=1.0),
        transition_duration=50,
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'])

    fig = go.Figure(data=data, layout=layout)
    return fig

## Pick a scenario, pick a portfolio and get the time series for a specific asset

In [ ]:
scenario = 'ssp585'
portfolio = 'US_ZIPCODES'
asset_index = 12

fig = display_timeseries_data(asset_index, scenario, portfolio)

fig.show()